<a href="https://colab.research.google.com/github/da-roth/DiffiPy/blob/main/DifferentiationInterfaceTest/examples-colab/aadc_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



In [ ]:
#Install DiffiPy
!pip install diffipy
import diffipy as dp

In [4]:
backend_array = ['numpy', 'torch',  'tensorflow', 'jax', 'aadc']

for backend in backend_array:
    dp.setBackend(backend)
    # Initialize variables and constants
    s0_value = 100.0
    K_value = 110.0
    r_value = 0.05
    sigma_value = 0.3
    dt_value = 1.0
    z_value = 0.5
    N = 1000000
    seed = 1

    # Define variables
    s0 = dp.variable(s0_value, 'input','s0')
    K = dp.variable(K_value, 'input','K')
    r = dp.variable(r_value, 'input','r')
    sigma = dp.variable(sigma_value, 'input','sigma')
    dt = dp.variable(dt_value, 'input','dt')
    z = dp.variable(z_value, 'randomVariableNormal','z')

    func_input_variables = [s0, K, r, sigma, dt, z]
    diff_variables = [s0, K, r, sigma, dt]

    # Record Tape: Standard Monte Carlo

    s = s0 * dp.exp((r - sigma **2 / 2) * dt + sigma * dp.sqrt(dt) * z)
    payoff =  dp.if_(s > K, s - K, 0)
    PV_standard = dp.exp(-r * dt) * dp.sum(payoff) / N

    ### Test performance
    dp.seed(seed)
    pre_computed_random_variables = z.NewSample(N) # pre-evaluate random samples

    PV_standard.run_performance_test(func_input_variables, diff_variables, warmup_iterations = 10, test_iterations = 40)


Backend              Eval-Result  mean runtime    variance runtime    gradient directions: {'r', 'K', 'dt', 's0', 'sigma'}
numpy                10.023520    0.057062        0.000044            [0.5002012963828406, -0.36360554602765655, 39.996882379789156, 39.89751398876251, 7.9844421986052785]
numpy_as_func        10.023520    0.060997        0.000028            {'s0': 0.5002013217847434, 'K': -0.36360553590242256, 'r': 39.99688241371757, 'sigma': 39.89751398751906, 'dt': 7.984442234842958}
torch                10.020878    0.007507        0.000000            [0.4992237389087677, -0.3627409040927887, 39.9015007019043, 39.90696334838867, 7.981120586395264]
torch_as_func        10.020875    0.001290        0.000000            [tensor(0.4992), tensor(-0.3627), tensor(39.9015), tensor(39.9070), tensor(7.9811)]
tensorflow           9.999600     0.080135        0.000023            [0.49935734, -0.36305577, 39.936134, 39.80618, 7.9677343]
tensorflow_as_func   9.999602     0.015833        0.00